# Simple Quantum Teleportation

Quantum Teleportation using the AWS bracket local simulator. This approach uses deferred measurement to calculate the fidelity of transportation at the end.

In [1]:
from braket.circuits import Circuit
from braket.devices import LocalSimulator

In [2]:
'''
teleport does something like so, e.g
init = |0>

1 = |0>
2 = |0>

# after bell-pair: 
1 == 2     ← they share (|00>+|11>)/√2

0 = |0>
1 = *      ← still entangled with 2


1 == 2     ← still entangled

# after corrections:
2 = |0>    ← teleported state, so q2 == q0_original
'''
def teleport(init='0'):
    circuit = Circuit()
    
    if init == '1':
        # we start at |0> so we flip to |1>
        circuit.x(0)

    # bell pair 1 & 2
    circuit.h(1)
    circuit.cnot(1, 2)
    
    # bell basis measurement 0 & 1    
    circuit.cnot(0, 1)
    circuit.h(0)
    
    # control x from qubit 1 to 2
    circuit.cnot(1, 2)

    # control z from qubit 0 to 2
    circuit.cz(0, 2)
    
    # measure it
    circuit.measure([2])
    return circuit

In [3]:
device = LocalSimulator()
shots = 1000

In [ ]:
# teleport |0> & |1> onto qubit 2
# on a noiseless sim we expect to see perfect fidelity
for state in ['0', '1']:
    circ = teleport(state)
    result = device.run(circ, shots=shots).result()
    
    counts = result.measurement_counts

    success = counts.get(state)
    fidelity = success / shots
    
    print(f"Teleport |{state}> → qubit 2: fidelity = {fidelity*100:.1f}%  counts = {counts}")

Teleport |0> → qubit 2: fidelity = 100.0%  counts = Counter({'0': 1000})
Teleport |1> → qubit 2: fidelity = 100.0%  counts = Counter({'1': 1000})
